In [183]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report


from keras.utils import np_utils

import tensorflow as tf
print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [112]:
def scale_x_encode_y(x, y):

    x_scaled = StandardScaler().fit_transform(x.values)
    
    
    encoder = LabelEncoder()
    encoder.fit(y)
    y_encoded = encoder.transform(y)
    y_encoded = np_utils.to_categorical(y_encoded)
    
    y_map = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))
 
    return(x_scaled, y_encoded, y_map)

In [113]:
df = pd.read_csv('data/IRIS.csv')

print(len(df))
df.head(10)

150


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [114]:
target_col = 'species'

x = df.loc[:, df.columns != target_col]
y = df[target_col]

In [ ]:
x_scaled, y_encoded, y_map = scale_x_encode_y(x, y)

In [121]:
train_x, valid_x, train_y, valid_y = train_test_split(x_scaled, y_encoded,train_size = 0.8,random_state = 1)

In [134]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(8, activation='relu', input_shape=[len(train_x[0])]),
  tf.keras.layers.Dense(3, activation='softmax')
])

In [135]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',    
    metrics=['accuracy'])

In [158]:
epoch_val         = 10
batch_size_val    = 10

verbose_val       = 2
workers_val       = -1

In [160]:
model.fit(x=x_scaled, 
          y=y_encoded, 
          epochs=epoch_val,
          batch_size = batch_size_val,
          validation_data = (valid_x, valid_y),
          workers= wokers_val,
         verbose=verbose_val)

Epoch 1/10
15/15 - 0s - loss: 0.1262 - accuracy: 0.9600 - val_loss: 0.1379 - val_accuracy: 0.9667
Epoch 2/10
15/15 - 0s - loss: 0.1246 - accuracy: 0.9600 - val_loss: 0.1352 - val_accuracy: 0.9667
Epoch 3/10
15/15 - 0s - loss: 0.1230 - accuracy: 0.9600 - val_loss: 0.1353 - val_accuracy: 0.9667
Epoch 4/10
15/15 - 0s - loss: 0.1220 - accuracy: 0.9667 - val_loss: 0.1331 - val_accuracy: 0.9667
Epoch 5/10
15/15 - 0s - loss: 0.1204 - accuracy: 0.9667 - val_loss: 0.1324 - val_accuracy: 0.9667
Epoch 6/10
15/15 - 0s - loss: 0.1195 - accuracy: 0.9667 - val_loss: 0.1286 - val_accuracy: 0.9667
Epoch 7/10
15/15 - 0s - loss: 0.1179 - accuracy: 0.9600 - val_loss: 0.1278 - val_accuracy: 0.9667
Epoch 8/10
15/15 - 0s - loss: 0.1166 - accuracy: 0.9733 - val_loss: 0.1273 - val_accuracy: 0.9667
Epoch 9/10
15/15 - 0s - loss: 0.1155 - accuracy: 0.9733 - val_loss: 0.1261 - val_accuracy: 0.9667
Epoch 10/10
15/15 - 0s - loss: 0.1146 - accuracy: 0.9733 - val_loss: 0.1256 - val_accuracy: 0.9667


In [180]:
predictions = model.predict(valid_x)
rounded_predictions = np.argmax(predictions, axis = -1)
rounded_valid_y = np.argmax(valid_y, axis = -1)

In [191]:
print(classification_report(rounded_valid_y, rounded_predictions, target_names = y_map.values()))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        11
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.86      1.00      0.92         6

       accuracy                           0.97        30
      macro avg       0.95      0.97      0.96        30
   weighted avg       0.97      0.97      0.97        30

